In [ ]:
!pip install faker
!pip install dnspython
!pip install tldextract
!pip install validators
!pip install whois
!pip install python-whois

In [ ]:
import pandas as pd
import urllib.request
import random
from faker import Faker
import requests
import dns.resolver
import tldextract
import ipaddress
import json
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import time
from google.colab import userdata
import re
from itertools import product
import validators
api_key = userdata.get('APIKEY')
cse_id = userdata.get('cseid')
similarweb_key = userdata.get('similarkey')
safe_browsing_key =  userdata.get('Safebrowsing')
uuid = userdata.get('urlscanapi')

fake = Faker()
user_agent = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36'

import whois

def check_whois_registered(domain):
    try:
        w = whois.whois(domain)
        # Instead of checking just domain_name, check for other fields as well
        if any([w.domain_name, w.registrar, w.creation_date, w.expiration_date]):
            return 1
    except whois.parser.PywhoisError as e:  # Adjusted for a more specific exception
        print(f"WHOIS lookup failed for {domain}: {e}")
    except Exception as e:  # Catching any other exception that might occur
        print(f"Unexpected error during WHOIS lookup for {domain}: {e}")
    return 0




def generate_random_urls_optimized(count=15000):
    fake = Faker()

    real_domains = [
        "google.com", "facebook.com", "amazon.com", "youtube.com", "wikipedia.org",
        "twitter.com", "linkedin.com", "instagram.com", "reddit.com", "pinterest.com"
    ]

    phishing_domains = [
        "google-security.com", "facebook-login.com", "amaz0n.co", "y0utube.com", "wikipediia.org",
        "tw1tter.com", "linkedinn.biz", "instagrarn.com", "redd1t.net", "pint3rest.com"
    ]

    deceptive_paths = ['/login/', '/verify/', '/secure/', '/update-account/', '/confirm/']
    queries = ['?username=admin&password=admin', '?user=guest&auth=1234', '?account=update&service=mail']

    real_paths = set()
    phishing_urls_set = set()

    # Generate unique real paths
    while len(real_paths) < count // 2:
        path = '/' + '/'.join(fake.words(nb=random.randint(1, 3)))
        real_paths.add(path)

    # Generate unique phishing URLs
    while len(phishing_urls_set) < count // 2:
        protocol = 'https://'
        domain = random.choice(phishing_domains)
        deceptive_path = random.choice(deceptive_paths)
        query = random.choice(queries)
        phishing_url = protocol + domain + deceptive_path + query
        phishing_urls_set.add((phishing_url, 'phishing'))

    # Convert sets to lists
    real_urls = [(f"https://{random.choice(real_domains)}{path}", 'real') for path in real_paths]
    phishing_urls = list(phishing_urls_set)

    # Shuffle both lists
    random.shuffle(real_urls)
    random.shuffle(phishing_urls)

    # Combine the two lists
    urls = real_urls + phishing_urls

    # Shuffle the combined list
    random.shuffle(urls)

    # Ensure the list has the correct count
    urls = urls[:count]

    # Ensure all URLs are unique and balance is maintained
    if len(urls) != len(set(urls)):
        raise ValueError("Duplicate URLs found in the generated list.")


    return urls



import socket
from urllib.parse import urlparse

def contains_ip(url):
    try:
        # Extract hostname from URL
        hostname = urlparse(url).hostname

        # Get the IP address for the hostname
        ip_address = socket.gethostbyname(hostname)

        return True
    except socket.gaierror:
        return False

def extract_features(url):
    suspicious_tlds = ['xyz', 'info', 'top', 'gq', 'cf', 'tk', 'ml', 'ga', 'men', 'loan', 'date', 'win', 'faith', 'review', 'party', 'webcam', 'trade', 'accountant', 'download', 'racing', 'science', 'cricket', 'bid']
    features = {}
    parsed_url = tldextract.extract(url)
    features['length'] = len(url)
    features['protocol'] = url.split('://')[0]
    features['domain'] = parsed_url.domain
    features['subdomain'] = parsed_url.subdomain
    features['suffix'] = parsed_url.suffix
    features['path'] = url.split(parsed_url.domain)[-1]
    features['number_of_subdomains'] = len(parsed_url.subdomain.split('.')) if parsed_url.subdomain else 0
    if contains_ip(url):
      features['has_ip_address'] = True
    else:
      features['has_ip_address'] = False
    features['is_https'] = features['protocol'] == 'https'
    features['special_char_count'] = sum(not c.isalnum() for c in url)
    suspicious_words = ['login', 'verify', 'account', 'secure', 'update', 'banking']
    features['has_suspicious_word'] = any(word in url for word in suspicious_words)
    features['is_suspicious_tld'] = features['suffix'] in suspicious_tlds
    return features

def get_dns_records(domain):
    records = {'A': [], 'MX': [], 'NS': []}
    try:
        a_records = dns.resolver.resolve(domain, 'A')
        records['A'] = [rdata.address for rdata in a_records]
    except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout, dns.exception.DNSException):
        records['A'] = 'Unavailable'
    try:
        mx_records = dns.resolver.resolve(domain, 'MX')
        records['MX'] = [rdata.exchange.to_text() for rdata in mx_records]
    except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout, dns.exception.DNSException):
        records['MX'] = 'Unavailable'
    try:
        ns_records = dns.resolver.resolve(domain, 'NS')
        records['NS'] = [rdata.to_text() for rdata in ns_records]
    except (dns.resolver.NXDOMAIN, dns.resolver.NoAnswer, dns.resolver.Timeout, dns.exception.DNSException):
        records['NS'] = 'Unavailable'
    return records



# import requests
# import os
# import time
# from bs4 import BeautifulSoup
# from urllib.parse import urlencode

# def check_google_indexing(url, user_agent):
#     headers = {'User-Agent': user_agent}
#     query = {'q': 'info:' + url}
#     google_url = "https://www.google.com/search?" + urlencode(query)

#     try:
#         response = requests.get(google_url, headers=headers)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Check if the URL is indexed
#         check = soup.find(id="rso").find("div", recursive=False).find("div", recursive=False).find("h3").find("a")
#         indexed = True if check else False
#         print(f"{url} is indexed: {indexed}")
#         return indexed
#     except requests.exceptions.HTTPError as e:
#         print(f"HTTP error occurred: {e}")
#         return False
#     except requests.exceptions.RequestException as e:
#         print(f"Other request error occurred: {e}")
#         return False

#     return indexed


import pandas as pd
import tldextract

def generate_and_collect_data(count=15000, include_suspicious=False, suspicious_ratio=0.3, api_key=None, cse_id=None, similarweb_key=None, safe_browsing_key=None, uuid=None):
    """
    Generate a dataset of URLs with associated features, labels, and WHOIS registration status.

    Parameters:
    - count (int): The number of URLs to generate.
    - include_suspicious (bool): Flag to include suspicious URLs.
    - suspicious_ratio (float): The proportion of suspicious URLs if included.
    - api_key, cse_id, similarweb_key, safe_browsing_key, uuid: API keys and IDs for various services.

    Returns:
    - DataFrame: A DataFrame containing the URL data with features, labels, and WHOIS registration status.
    """
    # Generate URLs with associated labels
    urls_with_labels = generate_random_urls_optimized(count)

    data = []
    for url, label in urls_with_labels:
        # Initialize the data dictionary for the URL
        url_data = {'URL': url, 'Label': label}

        # Extract URL features
        url_data.update(extract_features(url))

        # Parse the domain information
        parsed_url = tldextract.extract(url)
        full_domain = f"{parsed_url.subdomain}.{parsed_url.domain}.{parsed_url.suffix}".strip('.')

        # Add DNS records if the domain is valid
        if full_domain:
            url_data.update(get_dns_records(full_domain))

        # Check WHOIS registration status
        url_data['WHOIS_Registered'] = check_whois_registered(full_domain)

        # Additional API-based features can be added here

        # Append the data dictionary to the list
        data.append(url_data)

    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(data)
    return df



# Example usage with API keys
df = generate_and_collect_data(1500, include_suspicious=True, api_key=api_key, cse_id=cse_id, similarweb_key=similarweb_key, safe_browsing_key = safe_browsing_key, uuid=uuid)
df.to_csv('url_data.csv', index=False)
print(df)


KeyboardInterrupt: 